In [ ]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# train_df = train_df[train_df['Alkalinity, total'] < 650]
# train_df = train_df[train_df['Hardness'] < 1250]
# train_df = train_df[train_df['Total Dissolved Solids'] < 3200]
train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Chloride'] < 850]
train_df= train_df[train_df['Sulfate'] < 850]


# Display the first few rows of the training dataset to understand its structure
train_df.head(), train_df.columns

(   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
 0   0              120.0     24.0       6.9     0.314        7.1   
 1   1              140.0     41.0      11.0     0.150        6.5   
 2   2              160.0     51.0       9.0     0.450        9.1   
 3   3               46.0     13.0       2.9     0.120        2.7   
 4   4              240.0    147.0      70.0     0.600       48.0   
 
    Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
 0         0.498    48.0                  510.0     23.2   
 1         0.950    19.0                  300.0      8.5   
 2         2.500    12.0                  370.0     25.0   
 3         0.430     6.4                  110.0      5.0   
 4         6.800    93.0                 1410.0    418.0   
 
    Total Dissolved Solids  Hardness  
 0                   240.0      90.4  
 1                   190.0     130.0  
 2                   230.0     160.0  
 3                    97.0      41.0  
 4                   950.0 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
import pandas as pd

features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X = train_df[features]
y = train_df['Hardness']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best R^2 Score:", grid_search.best_score_)

best_rf_model = grid_search.best_estimator_
best_rf_model.fit(X_train, y_train)

y_val_pred = best_rf_model.predict(X_val)
val_r2_score = r2_score(y_val, y_val_pred)
print("Validation R^2 Score:", val_r2_score)

X_test = test_df[features]
y_test_pred = best_rf_model.predict(X_test)

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_test_pred})
print(predictions.head())

predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv_rf.csv', index=False)

Best Parameters: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best R^2 Score: 0.9081304706946263
Validation R^2 Score: 0.9171095753081348
     id    Hardness
0  6400   96.463076
1  6401  138.373883
2  6402  123.824775
3  6403   99.310574
4  6404  337.781131


In [ ]:
features

['Calcium', 'Magnesium']

XGB

In [ ]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Alkalinity, total'] < 650]
# train_df = train_df[train_df['Hardness'] < 1250]
# train_df = train_df[train_df['Total Dissolved Solids'] < 3200]
train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Chloride'] < 850]
train_df= train_df[train_df['Sulfate'] < 850]


# Display the first few rows of the training dataset to understand its structure
train_df.head(), train_df.columns

(   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
 0   0              120.0     24.0       6.9     0.314        7.1   
 1   1              140.0     41.0      11.0     0.150        6.5   
 2   2              160.0     51.0       9.0     0.450        9.1   
 3   3               46.0     13.0       2.9     0.120        2.7   
 4   4              240.0    147.0      70.0     0.600       48.0   
 
    Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
 0         0.498    48.0                  510.0     23.2   
 1         0.950    19.0                  300.0      8.5   
 2         2.500    12.0                  370.0     25.0   
 3         0.430     6.4                  110.0      5.0   
 4         6.800    93.0                 1410.0    418.0   
 
    Total Dissolved Solids  Hardness  
 0                   240.0      90.4  
 1                   190.0     130.0  
 2                   230.0     160.0  
 3                    97.0      41.0  
 4                   950.0 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

features = ['Calcium', 'Magnesium', 'Alkalinity, total', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

best_params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.8
}

model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

     id    Hardness
0  6400   85.261917
1  6401  164.413620
2  6402  110.107742
3  6403   98.174561
4  6404  329.539429


Code MODIFICATION FROM SUBMIT 8


In [ ]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]
# train_df = train_df[train_df['Hardness'] < 1000]
train_df = train_df[train_df['Chloride'] < 400]
# train_df= train_df[train_df['Sulfate'] < 650]

# Display the first few rows of the training dataset to understand its structure
train_df.head(), train_df.columns

(   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
 0   0              120.0     24.0       6.9     0.314        7.1   
 1   1              140.0     41.0      11.0     0.150        6.5   
 2   2              160.0     51.0       9.0     0.450        9.1   
 3   3               46.0     13.0       2.9     0.120        2.7   
 4   4              240.0    147.0      70.0     0.600       48.0   
 
    Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
 0         0.498    48.0                  510.0     23.2   
 1         0.950    19.0                  300.0      8.5   
 2         2.500    12.0                  370.0     25.0   
 3         0.430     6.4                  110.0      5.0   
 4         6.800    93.0                 1410.0    418.0   
 
    Total Dissolved Solids  Hardness  
 0                   240.0      90.4  
 1                   190.0     130.0  
 2                   230.0     160.0  
 3                    97.0      41.0  
 4                   950.0 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

best_params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.8
}


model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

     id    Hardness
0  6400   86.746201
1  6401  145.866257
2  6402  114.374176
3  6403   99.480301
4  6404  369.159851


DATE : 27 JUNE 2024


In [ ]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]
# train_df = train_df[train_df['Hardness'] < 1200]

# train_df = train_df[train_df['Total Dissolved Solids'] < 2000]

# Display the first few rows of the training dataset to understand its structure
train_df.head(), train_df.columns

(   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
 0   0              120.0     24.0       6.9     0.314        7.1   
 1   1              140.0     41.0      11.0     0.150        6.5   
 2   2              160.0     51.0       9.0     0.450        9.1   
 3   3               46.0     13.0       2.9     0.120        2.7   
 4   4              240.0    147.0      70.0     0.600       48.0   
 
    Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
 0         0.498    48.0                  510.0     23.2   
 1         0.950    19.0                  300.0      8.5   
 2         2.500    12.0                  370.0     25.0   
 3         0.430     6.4                  110.0      5.0   
 4         6.800    93.0                 1410.0    418.0   
 
    Total Dissolved Solids  Hardness  
 0                   240.0      90.4  
 1                   190.0     130.0  
 2                   230.0     160.0  
 3                    97.0      41.0  
 4                   950.0 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# creating ai model

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

# Features and target variable for training
# features = ['Calcium', 'Magnesium', 'Chloride', 'Total Dissolved Solids']
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X = train_df[features]
y = train_df['Hardness']


# X_test = test_df[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.8
}

model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

y_test_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Training MSE: {train_mse}")
print(f"Training R2: {train_r2}")
print(f"Testing MSE: {test_mse}")
print(f"Testing R2: {test_r2}")

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

Training MSE: 1921.1839944870708
Training R2: 0.9259669045654149
Testing MSE: 1857.9792331919639
Testing R2: 0.9292817222029351


ValueError: array length 1279 does not match index length 1601

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Filter the training data
train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]

# Display the first few rows of the training dataset to understand its structure
print(train_df.head())
print(train_df.columns)

from google.colab import drive
drive.mount('/content/drive')

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X = train_df[features]
y = train_df['Hardness']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Best parameters for the model
best_params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.8
}

# Create and train the model
model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Evaluate the model
r2 = r2_score(y_val, y_val_pred)
print(f'R2 Score on validation set: {r2}')

# Predict without evaluating since the test set does not have 'Hardness'
X_test = test_df[features]
y_test_pred = model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_test_pred})
print(predictions.head())

y_train_pred = model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

y_test_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Training MSE: {train_mse}")
print(f"Training R2: {train_r2}")
print(f"Testing MSE: {test_mse}")
print(f"Testing R2: {test_r2}")

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
0   0              120.0     24.0       6.9     0.314        7.1   
1   1              140.0     41.0      11.0     0.150        6.5   
2   2              160.0     51.0       9.0     0.450        9.1   
3   3               46.0     13.0       2.9     0.120        2.7   
4   4              240.0    147.0      70.0     0.600       48.0   

   Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
0         0.498    48.0                  510.0     23.2   
1         0.950    19.0                  300.0      8.5   
2         2.500    12.0                  370.0     25.0   
3         0.430     6.4                  110.0      5.0   
4         6.800    93.0                 1410.0    418.0   

   Total Dissolved Solids  Hardness  
0                   240.0      90.4  
1                   190.0     130.0  
2                   230.0     160.0  
3                    97.0      41.0  
4                   950.0     586.0  
Index(['

NameError: name 'y_test' is not defined

# MAINN CODE --> underfit


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Filter the training data
train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Total Dissolved Solids'] < 2000]


# Display the first few rows of the training dataset to understand its structure
print(train_df.head())
print(train_df.columns)

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Total Dissolved Solids']
# features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X = train_df[features]
y = train_df['Hardness']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Best parameters for the model
best_params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.8
}

# Create and train the model
model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Evaluate the model on the validation set
val_mse = mean_squared_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)
print(f'Validation MSE: {val_mse}')
print(f'Validation R2 Score: {val_r2}')

# Predict on the test set without evaluating (since the test set does not have 'Hardness')
X_test = test_df[features]
y_test_pred = model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_test_pred})
print(predictions.head())

# Evaluate on the training set (for completeness)
y_train_pred = model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
print(f'Training MSE: {train_mse}')
print(f'Training R2 Score: {train_r2}')

from google.colab import drive
drive.mount('/content/drive')
# Save predictions to CSV (optional)
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
0   0              120.0     24.0       6.9     0.314        7.1   
1   1              140.0     41.0      11.0     0.150        6.5   
2   2              160.0     51.0       9.0     0.450        9.1   
3   3               46.0     13.0       2.9     0.120        2.7   
4   4              240.0    147.0      70.0     0.600       48.0   

   Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
0         0.498    48.0                  510.0     23.2   
1         0.950    19.0                  300.0      8.5   
2         2.500    12.0                  370.0     25.0   
3         0.430     6.4                  110.0      5.0   
4         6.800    93.0                 1410.0    418.0   

   Total Dissolved Solids  Hardness  
0                   240.0      90.4  
1                   190.0     130.0  
2                   230.0     160.0  
3                    97.0      41.0  
4                   950.0     586.0  
Index(['

In [ ]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Calcium'] < 1000]
train_df = train_df[train_df['Magnesium'] < 150]
# train_df = train_df[train_df['Hardness'] < 1200]

# train_df = train_df[train_df['Total Dissolved Solids'] < 2000]

# Display the first few rows of the training dataset to understand its structure
train_df.head(), train_df.columns

import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
import pandas as pd

# Assuming train_df and test_df are already defined and loaded
# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

# Define the parameter grid
param_grid = {
    'colsample_bytree': 0.5,
    'learning_rate': 0.05,
    'max_depth': 3,
    'n_estimators': 200,
    'subsample': 0.8
}

# Train the model with the best parameters
best_model = xgb.XGBRegressor(**param_grid, objective='reg:squarederror')
best_model.fit(X_train, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = best_model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# Saving the predictions
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

     id    Hardness
0  6400   86.566368
1  6401  137.527878
2  6402  113.303574
3  6403   98.238945
4  6404  351.522919


In [ ]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Calcium'] < 500]
# train_df = train_df[train_df['Calcium'] < 450]
# train_df = train_df[train_df['Magnesium'] < 220]

# Display the first few rows of the training dataset to understand its structure
train_df.head(), train_df.columns

# creating ai model

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

best_params = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.8
}


model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

     id    Hardness
0  6400   86.384071
1  6401  143.537689
2  6402  114.700249
3  6403   98.293503
4  6404  360.310242


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

# Assuming train_df and test_df are already defined

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X = train_df[features]
y = train_df['Hardness']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Best parameters
best_params = {
    'colsample_bytree': 0.4,
    'learning_rate': 1.01,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.7,
    'alpha': 0.01,
    'gamma': 0.1,
    'reg_lambda': 1.1
}

# Define and train the model
model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
model.fit(X_train, y_train)

# Predict on training and validation sets
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)

# Evaluate the model
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Training R² score: {train_r2}")
print(f"Validation R² score: {val_r2}")

# Predict on test set (without evaluating, as test set does not have 'Hardness')
X_test = test_df[features]
y_test_pred = model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_test_pred})
print(predictions.head())

# Save predictions
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

Training R² score: 0.9212249974263669
Validation R² score: 0.9278513973333975
     id    Hardness
0  6400   81.636879
1  6401  143.538330
2  6402  115.029533
3  6403   92.605774
4  6404  370.455231


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
import pandas as pd

# Assuming train_df and test_df are already defined and loaded
# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

# Define the parameter grid
param_grid = {
    'colsample_bytree': 0.5,
    'learning_rate': 0.05,
    'max_depth': 3,
    'n_estimators': 200,
    'subsample': 0.8
}

# Initialize the model
# model = xgb.XGBRegressor(objective='reg:squarederror')

# best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')

# Perform grid search
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', cv=5, verbose=1)
# grid_search.fit(X_train, y_train)

# Best parameters from the grid search
# best_params = grid_search.best_params_
# print("Best parameters:", best_params)

# Train the model with the best parameters
best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
best_model.fit(X_train, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = best_model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# Saving the predictions
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


     id    Hardness
0  6400   85.517784
1  6401  145.626404
2  6402  119.001968
3  6403   99.723167
4  6404  344.828979


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
import pandas as pd

# Assuming train_df and test_df are already defined and loaded
# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

# Define the parameter grid
param_grid = {
    'colsample_bytree': [0.5, 0.7, 0.8],
    'learning_rate': [0.05, 0.1, 0.08],
    'max_depth': [3, 4, 5, 6],
    'n_estimators': [100, 200, 400, 600],
    'subsample': [0.7, 0.8, 0.9, 1.0]
}

# Initialize the model
model = xgb.XGBRegressor(objective='reg:squarederror')

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', cv=5, verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters from the grid search
best_params = grid_search.best_params_
print("Best parameters:", best_params)

# Train the model with the best parameters
best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
best_model.fit(X_train, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = best_model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# Saving the predictions
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


Fitting 5 folds for each of 576 candidates, totalling 2880 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
     id    Hardness
0  6400   89.737007
1  6401  132.751862
2  6402  123.900558
3  6403   99.909225
4  6404  329.936493


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X = train_df[features]
y = train_df['Hardness']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1, bootstrap=True, random_state=42)

rf_model.fit(X_train, y_train)

y_val_pred = rf_model.predict(X_val)

val_r2_score = r2_score(y_val, y_val_pred)
print("Validation R^2 Score:", val_r2_score)

X_test = test_df[features]

y_test_pred = rf_model.predict(X_test)

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_test_pred})
print(predictions.head())

predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv_rf.csv', index=False)

Validation R^2 Score: 0.9292967659902116
     id    Hardness
0  6400   98.052517
1  6401  145.226480
2  6402  125.923105
3  6403   97.364480
4  6404  342.218677


# 29 JUNE 2024

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]

print(train_df.head())
print(train_df.columns)

features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

best_params_xgb = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.8
}

best_params_rf = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}

model_xgb = xgb.XGBRegressor(objective='reg:squarederror', **best_params_xgb)
model_xgb.fit(X_train, y_train)

model_rf = RandomForestRegressor(**best_params_rf)
model_rf.fit(X_train, y_train)

y_pred_xgb = model_xgb.predict(X_test)
y_pred_rf = model_rf.predict(X_test)

y_pred_ensemble = (y_pred_xgb + y_pred_rf) / 2

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred_ensemble})
print(predictions.head())

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
0   0              120.0     24.0       6.9     0.314        7.1   
1   1              140.0     41.0      11.0     0.150        6.5   
2   2              160.0     51.0       9.0     0.450        9.1   
3   3               46.0     13.0       2.9     0.120        2.7   
4   4              240.0    147.0      70.0     0.600       48.0   

   Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
0         0.498    48.0                  510.0     23.2   
1         0.950    19.0                  300.0      8.5   
2         2.500    12.0                  370.0     25.0   
3         0.430     6.4                  110.0      5.0   
4         6.800    93.0                 1410.0    418.0   

   Total Dissolved Solids  Hardness  
0                   240.0      90.4  
1                   190.0     130.0  
2                   230.0     160.0  
3                    97.0      41.0  
4                   950.0     586.0  
Index(['

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]

print(train_df.head())
print(train_df.columns)

features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Original dataset shape:", X_train.shape)
print("Resampled dataset shape:", X_train_smote.shape)

best_params_xgb = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.1,
    'max_depth': 3,
    'n_estimators': 100,
    'subsample': 0.8
}

best_params_rf = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}

model_xgb = xgb.XGBRegressor(objective='reg:squarederror', **best_params_xgb)
model_xgb.fit(X_train_smote, y_train_smote)

model_rf = RandomForestRegressor(**best_params_rf)
model_rf.fit(X_train_smote, y_train_smote)

y_pred_xgb = model_xgb.predict(X_test)
y_pred_rf = model_rf.predict(X_test)

y_pred_ensemble = (y_pred_xgb + y_pred_rf) / 2

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred_ensemble})
print(predictions.head())

# For saving purpose
# predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
0   0              120.0     24.0       6.9     0.314        7.1   
1   1              140.0     41.0      11.0     0.150        6.5   
2   2              160.0     51.0       9.0     0.450        9.1   
3   3               46.0     13.0       2.9     0.120        2.7   
4   4              240.0    147.0      70.0     0.600       48.0   

   Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
0         0.498    48.0                  510.0     23.2   
1         0.950    19.0                  300.0      8.5   
2         2.500    12.0                  370.0     25.0   
3         0.430     6.4                  110.0      5.0   
4         6.800    93.0                 1410.0    418.0   

   Total Dissolved Solids  Hardness  
0                   240.0      90.4  
1                   190.0     130.0  
2                   230.0     160.0  
3                    97.0      41.0  
4                   950.0     586.0  
Index(['

ValueError: Unknown label type: 'continuous'

In [ ]:
import pandas as pd

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# train_df = train_df[train_df['Magnesium'] < 250]
# train_df = train_df[train_df['Calcium'] < 500]
# train_df = train_df[train_df['Calcium'] < 450]
# train_df = train_df[train_df['Magnesium'] < 220]

from sklearn.cluster import DBSCAN

db = DBSCAN(eps=20, min_samples=2).fit(train_df[['Magnesium']])

train_df['DBSCAN_Label'] = db.labels_
outliers = train_df[train_df['DBSCAN_Label'] == -1]

df_cleaned = train_df[train_df['DBSCAN_Label'] != -1].copy()

df_cleaned.drop(columns=['DBSCAN_Label'], inplace=True)

db = DBSCAN(eps=20, min_samples=2).fit(train_df[['Calcium']])

train_df['DBSCAN_Label'] = db.labels_
outliers = train_df[train_df['DBSCAN_Label'] == -1]

df_cleaned = train_df[train_df['DBSCAN_Label'] != -1].copy()

df_cleaned.drop(columns=['DBSCAN_Label'], inplace=True)

db = DBSCAN(eps=20, min_samples=2).fit(train_df[['Chloride']])

train_df['DBSCAN_Label'] = db.labels_
outliers = train_df[train_df['DBSCAN_Label'] == -1]

df_cleaned = train_df[train_df['DBSCAN_Label'] != -1].copy()

df_cleaned.drop(columns=['DBSCAN_Label'], inplace=True)

db = DBSCAN(eps=20, min_samples=2).fit(train_df[['Sulfate']])

train_df['DBSCAN_Label'] = db.labels_
outliers = train_df[train_df['DBSCAN_Label'] == -1]

df_cleaned = train_df[train_df['DBSCAN_Label'] != -1].copy()

df_cleaned.drop(columns=['DBSCAN_Label'], inplace=True)

# creating ai model

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

best_params = {
    'colsample_bytree': 0.749999901,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 99,
    'subsample': 0.8,
    'alpha': 0.009,
    'gamma': 0.12,
    'reg_lambda': 0.123456789,
     'min_child_weight': 1,
    'scale_pos_weight': 1
}


model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

     id    Hardness
0  6400   89.876068
1  6401  134.207092
2  6402  118.279968
3  6403   96.954872
4  6404  362.189728


# Code 61

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Calcium'] < 10000]

print(train_df.head(), train_df.columns)

features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("Explained variance ratio:", pca.explained_variance_ratio_)

best_params = {
    'colsample_bytree': 0.73,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 98,
    'subsample': 0.8,
    'alpha': 0.00001,
    'gamma': 0.12,
    'reg_lambda': 0.123456789
}

model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train_pca, y_train)

y_pred = model.predict(X_test_pca)

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)

   id  Alkalinity, total  Calcium  Chloride  Fluoride  Magnesium  \
0   0              120.0     24.0       6.9     0.314        7.1   
1   1              140.0     41.0      11.0     0.150        6.5   
2   2              160.0     51.0       9.0     0.450        9.1   
3   3               46.0     13.0       2.9     0.120        2.7   
4   4              240.0    147.0      70.0     0.600       48.0   

   Nitrate as N  Sodium  Specific Conductivity  Sulfate  \
0         0.498    48.0                  510.0     23.2   
1         0.950    19.0                  300.0      8.5   
2         2.500    12.0                  370.0     25.0   
3         0.430     6.4                  110.0      5.0   
4         6.800    93.0                 1410.0    418.0   

   Total Dissolved Solids  Hardness  
0                   240.0      90.4  
1                   190.0     130.0  
2                   230.0     160.0  
3                    97.0      41.0  
4                   950.0     586.0   Index(['

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

required_columns = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Hardness']
missing_columns_train = [col for col in required_columns if col not in train_df.columns]
missing_columns_test = [col for col in required_columns if col not in test_df.columns and col != 'Hardness']

if missing_columns_train:
    print(f"Missing columns in train_df: {missing_columns_train}")
if missing_columns_test:
    print(f"Missing columns in test_df: {missing_columns_test}")

molar_mass_calcium = 40.08
molar_mass_magnesium = 24.305

train_df['Mol Calcium'] = train_df['Calcium'] / molar_mass_calcium
train_df['Mol Magnesium'] = train_df['Magnesium'] / molar_mass_magnesium
train_df['Garam Karbonat (mol/L)'] = train_df['Mol Calcium'] + train_df['Mol Magnesium']
train_df['Garam Karbonat (mg/L)'] = train_df['Garam Karbonat (mol/L)'] * 100.0869

test_df['Mol Calcium'] = test_df['Calcium'] / molar_mass_calcium
test_df['Mol Magnesium'] = test_df['Magnesium'] / molar_mass_magnesium
test_df['Garam Karbonat (mol/L)'] = test_df['Mol Calcium'] + test_df['Mol Magnesium']
test_df['Garam Karbonat (mg/L)'] = test_df['Garam Karbonat (mol/L)'] * 100.0869

print(train_df[['Calcium', 'Magnesium', 'Garam Karbonat (mol/L)', 'Garam Karbonat (mg/L)']].head())

numeric_cols = train_df.select_dtypes(include= [np.number])
corr = numeric_cols.corr()
print('Most Correlated Feature with Hardness'), print(corr['Hardness'].sort_values(ascending = False),'\n')

features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Garam Karbonat (mg/L)']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

if 'id' not in test_df.columns:
    test_df['id'] = range(len(test_df))

best_params = {
    'colsample_bytree': 0.749999901,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 99,
    'subsample': 0.8,
    'alpha': 0.009,
    'gamma': 0.12,
    'reg_lambda': 0.123456789,
    'min_child_weight': 1,
    'scale_pos_weight': 1
}

model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


   Calcium  Magnesium  Garam Karbonat (mol/L)  Garam Karbonat (mg/L)
0     24.0        7.1                0.890923              89.169757
1     41.0        6.5                1.290389             129.151012
2     51.0        9.1                1.646864             164.829477
3     13.0        2.7                0.435440              43.581795
4    147.0       48.0                5.642567             564.747034
Most Correlated Feature with Hardness
Hardness                  1.000000
Total Dissolved Solids    0.815202
Sulfate                   0.658456
Alkalinity, total         0.639662
Specific Conductivity     0.635271
Chloride                  0.542279
Nitrate as N              0.286936
Garam Karbonat (mol/L)    0.053726
Garam Karbonat (mg/L)     0.053726
Mol Magnesium             0.044469
Magnesium                 0.044469
Mol Calcium               0.037651
Calcium                   0.037651
Sodium                    0.025808
Fluoride                  0.004647
id                     

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df = train_df[train_df['Calcium'] < 10000]
train_df = train_df[train_df['Magnesium'] < 250]

required_columns = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Hardness']
missing_columns_train = [col for col in required_columns if col not in train_df.columns]
missing_columns_test = [col for col in required_columns if col not in test_df.columns and col != 'Hardness']

if missing_columns_train:
    print(f"Missing columns in train_df: {missing_columns_train}")
if missing_columns_test:
    print(f"Missing columns in test_df: {missing_columns_test}")

molar_mass_calcium = 40.08
molar_mass_magnesium = 24.305

train_df['Mol Calcium'] = train_df['Calcium'] / molar_mass_calcium
train_df['Mol Magnesium'] = train_df['Magnesium'] / molar_mass_magnesium
train_df['Garam Karbonat (mol/L)'] = train_df['Mol Calcium'] + train_df['Mol Magnesium']
train_df['Garam Karbonat (mg/L)'] = train_df['Garam Karbonat (mol/L)'] * 100.0869

test_df['Mol Calcium'] = test_df['Calcium'] / molar_mass_calcium
test_df['Mol Magnesium'] = test_df['Magnesium'] / molar_mass_magnesium
test_df['Garam Karbonat (mol/L)'] = test_df['Mol Calcium'] + test_df['Mol Magnesium']
test_df['Garam Karbonat (mg/L)'] = test_df['Garam Karbonat (mol/L)'] * 100.0869

print(train_df[['Calcium', 'Magnesium', 'Garam Karbonat (mol/L)', 'Garam Karbonat (mg/L)']].head())

numeric_cols = train_df.select_dtypes(include= [np.number])
corr = numeric_cols.corr()
print('Most Correlated Feature with Hardness'), print(corr['Hardness'].sort_values(ascending = False),'\n')

features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Garam Karbonat (mg/L)']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

if 'id' not in test_df.columns:
    test_df['id'] = range(len(test_df))

best_params = {
    'colsample_bytree': 0.749999901,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 99,
    'subsample': 0.8,
    'alpha': 0.009,
    'gamma': 0.12,
    'reg_lambda': 0.123456789,
    'min_child_weight': 1,
    'scale_pos_weight': 1
}

model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


   Calcium  Magnesium  Garam Karbonat (mol/L)  Garam Karbonat (mg/L)
0     24.0        7.1                0.890923              89.169757
1     41.0        6.5                1.290389             129.151012
2     51.0        9.1                1.646864             164.829477
3     13.0        2.7                0.435440              43.581795
4    147.0       48.0                5.642567             564.747034
Most Correlated Feature with Hardness
Hardness                  1.000000
Garam Karbonat (mol/L)    0.960100
Garam Karbonat (mg/L)     0.960100
Mol Calcium               0.876970
Calcium                   0.876970
Magnesium                 0.833467
Mol Magnesium             0.833467
Total Dissolved Solids    0.818672
Sulfate                   0.659454
Alkalinity, total         0.640111
Specific Conductivity     0.634806
Chloride                  0.541444
Nitrate as N              0.287476
Sodium                    0.085338
Fluoride                  0.004773
id                     

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')



required_columns = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Hardness']
missing_columns_train = [col for col in required_columns if col not in train_df.columns]
missing_columns_test = [col for col in required_columns if col not in test_df.columns and col != 'Hardness']

if missing_columns_train:
    print(f"Missing columns in train_df: {missing_columns_train}")
if missing_columns_test:
    print(f"Missing columns in test_df: {missing_columns_test}")

molar_mass_calcium = 40.08
molar_mass_magnesium = 24.305

train_df['Mol Calcium'] = train_df['Calcium'] / molar_mass_calcium
train_df['Mol Magnesium'] = train_df['Magnesium'] / molar_mass_magnesium
train_df['Garam Karbonat (mol/L)'] = train_df['Mol Calcium'] + train_df['Mol Magnesium']
train_df['Garam Karbonat (mg/L)'] = train_df['Garam Karbonat (mol/L)'] * 100.0869

test_df['Mol Calcium'] = test_df['Calcium'] / molar_mass_calcium
test_df['Mol Magnesium'] = test_df['Magnesium'] / molar_mass_magnesium
test_df['Garam Karbonat (mol/L)'] = test_df['Mol Calcium'] + test_df['Mol Magnesium']
test_df['Garam Karbonat (mg/L)'] = test_df['Garam Karbonat (mol/L)'] * 100.0869

print(train_df[['Calcium', 'Magnesium', 'Garam Karbonat (mol/L)', 'Garam Karbonat (mg/L)']].head())

numeric_cols = train_df.select_dtypes(include= [np.number])
corr = numeric_cols.corr()
print('Most Correlated Feature with Hardness'), print(corr['Hardness'].sort_values(ascending = False),'\n')

features = ['Chloride', 'Sulfate', 'Garam Karbonat (mg/L)']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

if 'id' not in test_df.columns:
    test_df['id'] = range(len(test_df))

best_params = {
    'colsample_bytree': 0.749999901,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 99,
    'subsample': 0.8,
    'alpha': 0.009,
    'gamma': 0.12,
    'reg_lambda': 0.123456789,
    'min_child_weight': 1,
    'scale_pos_weight': 1
}

model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# for saving purpose
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


   Calcium  Magnesium  Garam Karbonat (mol/L)  Garam Karbonat (mg/L)
0     24.0        7.1                0.890923              89.169757
1     41.0        6.5                1.290389             129.151012
2     51.0        9.1                1.646864             164.829477
3     13.0        2.7                0.435440              43.581795
4    147.0       48.0                5.642567             564.747034
Most Correlated Feature with Hardness
Hardness                  1.000000
Total Dissolved Solids    0.815202
Sulfate                   0.658456
Alkalinity, total         0.639662
Specific Conductivity     0.635271
Chloride                  0.542279
Nitrate as N              0.286936
Garam Karbonat (mol/L)    0.053726
Garam Karbonat (mg/L)     0.053726
Mol Magnesium             0.044469
Magnesium                 0.044469
Mol Calcium               0.037651
Calcium                   0.037651
Sodium                    0.025808
Fluoride                  0.004647
id                     

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Filter outliers
train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Calcium'] < 500]

# Add new features based on molar mass
molar_mass_calcium = 40.08
molar_mass_magnesium = 24.305

train_df['Mol Calcium'] = train_df['Calcium'] / molar_mass_calcium
train_df['Mol Magnesium'] = train_df['Magnesium'] / molar_mass_magnesium
train_df['Garam Karbonat (mol/L)'] = train_df['Mol Calcium'] + train_df['Mol Magnesium']
train_df['Garam Karbonat (mg/L)'] = train_df['Garam Karbonat (mol/L)'] * 100.0869

test_df['Mol Calcium'] = test_df['Calcium'] / molar_mass_calcium
test_df['Mol Magnesium'] = test_df['Magnesium'] / molar_mass_magnesium
test_df['Garam Karbonat (mol/L)'] = test_df['Mol Calcium'] + test_df['Mol Magnesium']
test_df['Garam Karbonat (mg/L)'] = test_df['Garam Karbonat (mol/L)'] * 100.0869

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Garam Karbonat (mol/L)', 'Garam Karbonat (mg/L)']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("Explained variance ratio:", pca.explained_variance_ratio_)

# Parameters for XGBoost
best_params = {
    'colsample_bytree': 0.73,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 98,
    'subsample': 0.8,
    'alpha': 0.00001,
    'gamma': 0.12,
    'reg_lambda': 0.123456789
}

# Create and train the model
model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train_pca, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = model.predict(X_test_pca)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# Save predictions
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


Explained variance ratio: [0.74091057 0.11866145]
     id    Hardness
0  6400   92.581650
1  6401  212.703201
2  6402  123.452110
3  6403   94.715683
4  6404  353.568207


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Filter outliers
train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Calcium'] < 500]

# Add new features based on molar mass
molar_mass_calcium = 40.08
molar_mass_magnesium = 24.305

train_df['Mol Calcium'] = train_df['Calcium'] / molar_mass_calcium
train_df['Mol Magnesium'] = train_df['Magnesium'] / molar_mass_magnesium
train_df['Garam Karbonat (mol/L)'] = train_df['Mol Calcium'] + train_df['Mol Magnesium']
train_df['Garam Karbonat (mg/L)'] = train_df['Garam Karbonat (mol/L)'] * 100.0869

test_df['Mol Calcium'] = test_df['Calcium'] / molar_mass_calcium
test_df['Mol Magnesium'] = test_df['Magnesium'] / molar_mass_magnesium
test_df['Garam Karbonat (mol/L)'] = test_df['Mol Calcium'] + test_df['Mol Magnesium']
test_df['Garam Karbonat (mg/L)'] = test_df['Garam Karbonat (mol/L)'] * 100.0869

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Garam Karbonat (mol/L)', 'Garam Karbonat (mg/L)']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("Explained variance ratio:", pca.explained_variance_ratio_)

# Parameters for XGBoost
best_params = {
    'colsample_bytree': 0.73,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 98,
    'subsample': 0.8,
    'alpha': 0.00001,
    'gamma': 0.12,
    'reg_lambda': 0.123456789
}

# Create and train the model
model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train_pca, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = model.predict(X_test_pca)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# Save predictions
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


Explained variance ratio: [0.74091057 0.11866145]
     id    Hardness
0  6400   92.581650
1  6401  212.703201
2  6402  123.452110
3  6403   94.715683
4  6404  353.568207


In [ ]:
import pandas as pd
import xgboost as xgb

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Filter outliers
train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Calcium'] < 500]

# Add new features based on molar mass
molar_mass_calcium = 40.08
molar_mass_magnesium = 24.305

train_df['Mol Calcium'] = train_df['Calcium'] / molar_mass_calcium
train_df['Mol Magnesium'] = train_df['Magnesium'] / molar_mass_magnesium
train_df['Garam Karbonat (mol/L)'] = train_df['Mol Calcium'] + train_df['Mol Magnesium']
train_df['Garam Karbonat (mg/L)'] = train_df['Garam Karbonat (mol/L)'] * 100.0869

test_df['Mol Calcium'] = test_df['Calcium'] / molar_mass_calcium
test_df['Mol Magnesium'] = test_df['Magnesium'] / molar_mass_magnesium
test_df['Garam Karbonat (mol/L)'] = test_df['Mol Calcium'] + test_df['Mol Magnesium']
test_df['Garam Karbonat (mg/L)'] = test_df['Garam Karbonat (mol/L)'] * 100.0869

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Garam Karbonat (mol/L)', 'Garam Karbonat (mg/L)']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

# Parameters for XGBoost
best_params = {
    'colsample_bytree': 0.73,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 98,
    'subsample': 0.8,
    'alpha': 0.00001,
    'gamma': 0.12,
    'reg_lambda': 0.123456789
}

# Create and train the model
model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = model.predict(X_test)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# Save predictions
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


     id    Hardness
0  6400   99.014023
1  6401  140.002106
2  6402  121.072144
3  6403   98.751053
4  6404  339.596924


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Filter outliers
train_df = train_df[train_df['Magnesium'] < 250]
train_df = train_df[train_df['Calcium'] < 500]

# Add new features based on molar mass
molar_mass_calcium = 40.08
molar_mass_magnesium = 24.305

train_df['Mol Calcium'] = train_df['Calcium'] / molar_mass_calcium
train_df['Mol Magnesium'] = train_df['Magnesium'] / molar_mass_magnesium
train_df['Garam Karbonat (mol/L)'] = train_df['Mol Calcium'] + train_df['Mol Magnesium']
train_df['Garam Karbonat (mg/L)'] = train_df['Garam Karbonat (mol/L)'] * 100.0869

test_df['Mol Calcium'] = test_df['Calcium'] / molar_mass_calcium
test_df['Mol Magnesium'] = test_df['Magnesium'] / molar_mass_magnesium
test_df['Garam Karbonat (mol/L)'] = test_df['Mol Calcium'] + test_df['Mol Magnesium']
test_df['Garam Karbonat (mg/L)'] = test_df['Garam Karbonat (mol/L)'] * 100.0869

# Features and target variable for training
features = ['Calcium', 'Magnesium', 'Chloride', 'Sulfate', 'Garam Karbonat (mol/L)', 'Garam Karbonat (mg/L)']
X_train = train_df[features]
y_train = train_df['Hardness']

X_test = test_df[features]

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Parameters for XGBoost
best_params = {
    'colsample_bytree': 0.73,
    'learning_rate': 0.116,
    'max_depth': 3,
    'n_estimators': 98,
    'subsample': 0.8,
    'alpha': 0.00001,
    'gamma': 0.12,
    'reg_lambda': 0.123456789
}

# Create and train the model
model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train_scaled, y_train)

# Predict without evaluating since the test set does not have 'Hardness'
y_pred = model.predict(X_test_scaled)

# Displaying predictions
predictions = pd.DataFrame({'id': test_df['id'], 'Hardness': y_pred})
print(predictions.head())

# Save predictions
predictions.to_csv('drive/My Drive/Colab Notebooks/centerdiv.csv', index=False)


     id    Hardness
0  6400   99.014023
1  6401  140.002106
2  6402  121.072144
3  6403   98.751053
4  6404  339.596924
